# MIDI - Music

Das Midi-Format wurde 1981 definiert und ist seitdem unverändert geblieben. Es ist sehr simpel gehalten; Geräte emfangen und senden MidiMessages, die

Die wichtigsten Midi-Befehle sind natürlich die Anweisungen zum Abspielen eines Tons. Dabei sind zum Abspielen einer Note zwei Nachrichten notwendig: eine zum Starten und eine zum Beenden des Tons. Es gibt keine einzelne Anweisung "Spiele einen Ton für eine bestimmte Zeitdauer".
Verständich ist dieses Verhalten, wenn man an die "Eingabe" einer Note über eine angeschlossene Tastatut denkt. Ein Note wird angeschlagen, was zu einem Midi-Ereignis führt, bestehend aus den Daten

* Taste wurde gedrückt (Art des Ereignisses, "type=note_on")
* Zeitpunkt des Ereignisses nach dem letzten Ereignis ("time")
* Angeschlagene Taste ("note=0-128")
* Geschwindigkeit des Anschlags, falls es sich um eine anschlagssensitive Tastatur handelt ("velocity=80")

Wenn die Taste losgelassen wird, entsteht ein weiteres Ereignis

* Taste wurde losgelassen ("type=note_off")
* Zeitpunkt des Ereignisses nach dem letzten Ereignis ("time="0.5")
* Losgelassene Taste ("note=0-128")
* Geschwindigkeit des Anschlags ("velocity=0")

Beide Ereigniss zusammen ergeben die gespielte Note.

## Tempo

In der Musik ist die Viertelnote das gebräuchliche Zählmaß. Man zählt 1, 2, 3, 4, 1, 2, 3, 4,... für einen Vier-Vierteltakt und 1, 2, 3, 1, 2, 3,... für den Drei-Vierteltakt / Walzertakt. Diese Vierteltnote, im englischen der __Beat__, gibt das Tempo vor. Gebrüchlich sind etwa 120 __bpm__, also 120 __Beats per Minute__, was bedeutet, dass es 120 Vieteltakte in der Minute gibt, also pro Sekunde zwei Vierteltakte. Ein Vierteltakt ist dan 0,5 Sekunden lang.

Die zeit in Midi wird in Mikroskunden gemessen. Zwischen zwei Midi-Ereignissen vergeht dabei eine gewissen Anzahl an Mikrosekunden. Wenn eine Note eine Viertelnote lang sein soll, dann vergeht bei e120 bpm eine halbe Sekunde und damit 500.000 Mikrosekunden. In dieser Größenordnung bewegen sich also die Werte für das Tempo, und 500.000 ist ein guter Standardwert für das Temp, das dann 120 bpm (Moderato / Allegro) entspricht.

Um eine Viertelnote oder einen Note mit einem anderen Notenwert abzuspielen, muss ihr Wert dann auf diese Ticks oder auf Mikrosekunden umgerechnet werden. 

midi_header = 0x4d546864
header_chunk = "MThd" + <header_length> + <format> + <n> + <division>
"MThd" 4 bytes
the literal string MThd, or in hexadecimal notation: 0x4d546864. These four characters at the start of the MIDI file indicate that this is a MIDI file.
<header_length> 4 bytes
length of the header chunk (always 6 bytes long&emdash;the size of the next three fields).
<format> 2 bytes
0 = single track file format
1 = multiple track file format
2 = multiple song file format
<n> 2 bytes
number of tracks that follow
<division> 2 bytes
unit of time for delta timing. If the value is positive, then it represents the units per beat. 
For example, +96 would mean 96 ticks per beat. If the value is negative, delta times are in SMPTE compatible units.

In [1]:
import mido
from mido import Message,MetaMessage,MidiFile
import time

In [68]:

DEFAULT_TEMPO = 500000
DEFAULT_TICKS_PER_BEAT = 480

def get_midi_port_names():
    return mido.get_output_names()
    
def get_midi_port(number):
    port = get_midi_port_names()[number]
    return mido.open_output(port)

def play_tune(tune):
    '''
    Spiel eine Song in Form eines formatierten Strings.
    '''
    messages = parse_tune(tune)
    play_messages(messages)
    
    
def play_messages(messages):    
    '''
    Spielt eine Sequenz von MIDI-Messages ab.
    '''
    with get_midi_port(0) as out:
        for msg in get_messages(messages):
            out.send(msg)
            
def get_messages(messages):
    tempo = DEFAULT_TEMPO
    ticks_per_beat=DEFAULT_TICKS_PER_BEAT
    for msg in get_next_message(messages):
        if msg.time > 0:
            delta = tick2second(msg.time, ticks_per_beat, tempo)
        else:
            delta = 0

        yield msg.copy(time=delta)

        if msg.type == 'set_tempo':
            tempo = msg.tempo     
            

def tick2second(tick, ticks_per_beat, tempo):
    scale = tempo * 1e-6 / ticks_per_beat
    return tick * scale

    
def get_next_message(messages,now=time.time):   
    start_time = now()
    input_time = 0.0

    for msg in messages:
        input_time += msg.time
        playback_time = now() - start_time
        duration_to_next_event = input_time - playback_time

        if duration_to_next_event > 0.0:
            time.sleep(duration_to_next_event)

        if isinstance(msg, MetaMessage):
            continue
        else:
            yield msg
    return

names = "c c# d d# e f f# g g# a a# h C C# D D# E F F# G G# A A# H".split(' ')
midi_map = dict(zip(names,[v for v in range(60,85)]))
midi_map['r']=0

midi_map_rev = {v: k for k, v in midi_map.items()}
print(midi_map_rev)

def pitch2midi(pitch):
    return midi_map[pitch]



{60: 'c', 61: 'c#', 62: 'd', 63: 'd#', 64: 'e', 65: 'f', 66: 'f#', 67: 'g', 68: 'g#', 69: 'a', 70: 'a#', 71: 'h', 72: 'C', 73: 'C#', 74: 'D', 75: 'D#', 76: 'E', 77: 'F', 78: 'F#', 79: 'G', 80: 'G#', 81: 'A', 82: 'A#', 83: 'H', 0: 'r'}


In [60]:

c4 = Note.parse_midi(1)
print(c4)

Note(pitch=c, value=4, dotted=True)


In [71]:
class Note(object):

    time = 4 * 480
        
    def __init__(self,pitch,value=4,dotted = False,volume=80):
        self.pitch = pitch
        self.value = value
        self.dotted = dotted
        self.volume = volume

    @classmethod    
    def parse_text(self,text):
        '''
        Generates values for pitch, value and dotted from a string.
        Durch regulären Ausdruck ersetzen
        '''
        tokens = text.strip().split('.')
        dotted = len(tokens) > 1
        text = tokens[0]
     
        pitch = ''.join([char for char in text if (not char.isdigit())])
        value = ''.join([char for char in text if (char.isdigit())])
        value = int(value)
     
        return Note(pitch, value, dotted)
    
    def compute_midi_time(self):
        '''
        Berechnet die Dauer der Note.
        '''
        return int((1.5 * Note.time / self.value)) if self.dotted else int((Note.time / self.value)) 
    
    def midi_messages(self):
        
        time = self.compute_midi_time()
        if self.is_pause():
            return  [Message('note_on',note=pitch2midi(self.pitch),velocity=0,time= time)]
        else:
            return  [Message('note_on',note=pitch2midi(self.pitch),velocity= self.volume,time=0),
                     Message('note_on',note=pitch2midi(self.pitch),velocity= 0,time= time)]          
        
    def is_pause(self):
        return self.pitch == 'r'  
    
               
    def dict(self):
        return {'pitch' : self.pitch, 'value' : self.value, 'dotted' : self.dotted, 'volume' : self.volume}
    
    def __str__(self):
        return "Note(pitch=" + self.pitch + ", value=" + str(self.value) + ", dotted=" + str(self.dotted) +")"
    
    @classmethod
    def parse_midi(self,msgs):
        '''
        Factory method. Construct notes from Midi messages.
        '''
        msg = msgs[1]
        return Note(midi_map_rev[msg.note],Note.time / msg.time)    
    


In [73]:
msgs = [Message('note_on', channel=0, note=67, velocity=80, time=0),
    Message('note_on', channel=0, note=67, velocity=0, time=960)]

note = Note.parse_midi(msgs)

print(note)

Note(pitch=g, value=2.0, dotted=False)


In [ ]:
    
class Tune(object):
    def __init__(self,notes,volume=80,track=0):
        notes = [note for note in notes.replace('|','').split(' ') if len(note) > 1] 
        self.notes = [Note.parse_text(note) for note in notes]
        self.volume = volume
    
    def midi_messages(self):
        messages = []
        for note in self.notes:
            print(note)
            for message in note.midi_messages():
                messages.append( message)
           
        return messages
    
    def midi(self):
        '''
        Returns the Midi representation of the tune as a MidiFile object. 
        This object can be directly playes or saved as a file.
        '''
        midi = MidiFile()
        track = mido.MidiTrack() 
        midi.tracks.append(track)
        for msg in self.midi_messages():
            track.append(msg)           
        return midi    
    
    def play(self,port):
        '''
        Plays the midi representation of the tune at the specified port.
        '''
        with port as p:
            for msg in self.midi().play():
                p.send(msg)

In [61]:
haenschen_klein = Tune("g4 e4 e4 r4  | f4 d4 d4 r4  |  || c4 d4 e4 f4 | g4 g4 g4 r4")
print(haenschen_klein.midi())
haenschen_klein.play(get_midi_port(0))
jingle_bells = Tune("e4 e4 e2 | e4 e4 e2  | e4 g4 c4 d4 | e1 | f4 f4 f4 f4 |  f4 e4 e2 | e4 d4 d4 e4 | d2 g2 |  e4 e4 e2 | e4 e4 e2 | e4 g4 c4 d4 | e1 | f4 f4 f4 f4 | f4 e4 e4 e4 | g4 g4 f4 d4 | c1")



MidiFile(type=1, ticks_per_beat=480)
Note(pitch=g, value=4, dotted=False)
Note(pitch=e, value=4, dotted=False)
Note(pitch=e, value=4, dotted=False)
Note(pitch=r, value=4, dotted=False)
Note(pitch=f, value=4, dotted=False)
Note(pitch=d, value=4, dotted=False)
Note(pitch=d, value=4, dotted=False)
Note(pitch=r, value=4, dotted=False)
Note(pitch=c, value=4, dotted=False)
Note(pitch=d, value=4, dotted=False)
Note(pitch=e, value=4, dotted=False)
Note(pitch=f, value=4, dotted=False)
Note(pitch=g, value=4, dotted=False)
Note(pitch=g, value=4, dotted=False)
Note(pitch=g, value=4, dotted=False)
Note(pitch=r, value=4, dotted=False)
MidiFile(type=1, ticks_per_beat=480, tracks=[
  MidiTrack([
    Message('note_on', channel=0, note=67, velocity=80, time=0),
    Message('note_on', channel=0, note=67, velocity=0, time=480),
    Message('note_on', channel=0, note=64, velocity=80, time=0),
    Message('note_on', channel=0, note=64, velocity=0, time=480),
    Message('note_on', channel=0, note=64, veloci

### Einige Musikstücke laden

In [6]:
#persuaders = MidiFile('midi/ThePersuaders.mid')
#heiko = MidiFile('midi/Heiko.mid')
#troubadour = MidiFile('midi/Der_Troubadour.mid')
heiko = MidiFile('midi/Heiko.mid')
print(heiko)

MidiFile(type=1, ticks_per_beat=480, tracks=[
  MidiTrack([
    MetaMessage('track_name', name='Klavier', time=0),
    MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0),
    MetaMessage('key_signature', key='C', time=0),
    MetaMessage('set_tempo', tempo=500000, time=0),
    Message('control_change', channel=0, control=121, value=0, time=0),
    Message('control_change', channel=0, control=100, value=0, time=0),
    Message('control_change', channel=0, control=101, value=0, time=0),
    Message('control_change', channel=0, control=6, value=12, time=0),
    Message('control_change', channel=0, control=100, value=127, time=0),
    Message('control_change', channel=0, control=101, value=127, time=0),
    Message('program_change', channel=0, program=0, time=0),
    Message('control_change', channel=0, control=7, value=100, time=0),
    Message('control_change', channel=0, control=10, value=64, time=0),
    Message('contro